In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

In [81]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet('./' + save_name + '.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [82]:
# pyarrow 설치 해야함, 9.0.0
csv_to_parquet('C:/Users/knzss/Documents/ds_study/project/ds5-ml-repo-1/data/train.csv', 'train')
csv_to_parquet('C:/Users/knzss/Documents/ds_study/project/ds5-ml-repo-1/data/test.csv', 'test')

train Done.
test Done.


In [83]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [84]:
train

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,1,106,지방도1112호선,0,0,60.0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,2,103,일반국도11호선,0,0,60.0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,2,103,일반국도16호선,0,0,80.0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,2,107,태평로,0,0,50.0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,TRAIN_4701212,20211104,목,16,1,107,-,0,0,50.0,...,0,대림사거리,33.422145,126.278125,없음,금덕해운,33.420955,126.273750,없음,20.0
4701213,TRAIN_4701213,20220331,목,2,2,107,-,0,0,80.0,...,3,광삼교,33.472505,126.424368,없음,광삼교,33.472525,126.424890,없음,65.0
4701214,TRAIN_4701214,20220613,월,22,2,103,일반국도12호선,0,0,60.0,...,0,고성교차로,33.447183,126.912579,없음,성산교차로,33.444121,126.912948,없음,30.0
4701215,TRAIN_4701215,20211020,수,2,2,103,일반국도95호선,0,0,80.0,...,0,제6광령교,33.443596,126.431817,없음,관광대학입구,33.444996,126.433332,없음,73.0


In [ ]:
# 전처리

In [86]:
# [요일, 시작 지점의 회전제한 유무, 도착지점의 회전제한 유무]
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
            
    print(i, le.classes_)
    test[i]=le.transform(test[i])

C:\Users\knzss\AppData\Local\Temp\ipykernel_18940\4181031811.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if label not in le.classes_:


day_of_week ['0' '1' '2' '3' '4' '5' '6' '금' '목' '수' '월' '일' '토' '화']
start_turn_restricted ['없음' '있음']
end_turn_restricted ['없음' '있음']


In [ ]:
y_train = train['target'] 

# [아이디, 날짜, 평균속도, 도로명, 시작지점명, 도착지점명, 통과제한차량]
X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(test.shape)

주말, 공휴일?, 연후 시작일, 중간일, 종료일
https://nomalcy.tistory.com/303

In [21]:
X_train = train.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

In [30]:
# 요일, 시간대, 차로수, 도로등급, 중용구간 여부, 연결로 코드, 최고속도제한, 통과제한하중, 통과제한높이
# 도로유형, 시작지점위도, 시작지점경도, 시작지점의 회전제한 유무
# 도착지점위도, 도착지점경도, 도착지점의 회전제한 유무
# 16개
X_train.head(7)

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,1,17,1,106,0,0,60.0,32400.0,0.0,3,33.427747,126.662612,0,33.427749,126.662335,0
1,1,21,2,103,0,0,60.0,0.0,0.0,0,33.500730,126.529107,1,33.504811,126.526240,0
2,4,7,2,103,0,0,80.0,0.0,0.0,0,33.279145,126.368598,0,33.280072,126.362147,0
3,0,13,2,107,0,0,50.0,0.0,0.0,0,33.246081,126.567204,0,33.245565,126.566228,0
4,6,8,2,103,0,0,80.0,0.0,0.0,0,33.462214,126.326551,0,33.462677,126.330152,0
5,3,7,2,107,0,0,60.0,0.0,0.0,0,33.249949,126.505664,0,33.252183,126.506069,0
6,1,0,2,107,0,0,60.0,0.0,0.0,0,33.418412,126.268029,0,33.414175,126.269378,0


In [26]:
# 7월 데이터를 빼서 테스트데이터로 사용하면 좋을듯
train.sort_values('base_date')

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
430379,TRAIN_0430379,20210901,2,16,3,106,지방도1132호선,0,0,70.0,...,3,진동1로,33.518987,126.568861,1,동중입구,33.519726,126.571488,0,37.0
369019,TRAIN_0369019,20210901,2,11,1,103,일반국도16호선,0,0,50.0,...,0,모살물교,33.312509,126.703863,0,한남교,33.312449,126.696173,0,36.0
1132647,TRAIN_1132647,20210901,2,4,2,103,일반국도12호선,0,0,80.0,...,0,애월샷시,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,43.0
4543127,TRAIN_4543127,20210901,2,16,1,107,-,0,0,60.0,...,0,서중2교,33.331742,126.676431,0,한남교차로,33.325096,126.665698,0,55.0
3428956,TRAIN_3428956,20210901,2,9,1,103,일반국도16호선,0,0,50.0,...,3,월산2교,33.260056,126.490987,0,월산2교,33.259995,126.491306,0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053638,TRAIN_2053638,20220731,4,21,1,107,중문로,0,0,50.0,...,0,신양약방,33.251716,126.425612,0,중문한의원,33.251884,126.423200,0,21.0
2748124,TRAIN_2748124,20220731,4,12,3,107,연삼로,0,0,70.0,...,0,제2거로교,33.508464,126.558231,0,거로사거리,33.507349,126.556173,1,36.0
2447938,TRAIN_2447938,20220731,4,12,2,107,경찰로,0,0,60.0,...,0,신성교회,33.253074,126.506393,0,서호2차현대맨션203동,33.252183,126.506069,0,24.0
3749761,TRAIN_3749761,20220731,4,1,2,103,일반국도16호선,0,0,60.0,...,0,서홍교,33.264112,126.554043,0,서흥동사무소,33.263685,126.550978,0,37.0


In [27]:
test.head(10)

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,height_restricted,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted
0,TEST_000000,20220825,1,17,3,107,연삼로,0,0,70.0,...,0.0,0,산지2교,33.499427,126.541298,0,제주은행사거리,33.500772,126.543837,1
1,TEST_000001,20220809,6,12,2,103,일반국도12호선,0,0,70.0,...,0.0,3,중문입구,33.258507,126.427003,0,관광단지입구,33.258119,126.415840,0
2,TEST_000002,20220805,0,2,1,103,일반국도16호선,0,0,60.0,...,0.0,0,도순3교,33.258960,126.476508,0,도순2교,33.259206,126.474687,0
3,TEST_000003,20220818,1,23,3,103,일반국도11호선,0,0,70.0,...,0.0,0,아라주공아파트,33.473494,126.545647,0,인다마을,33.471061,126.545467,0
4,TEST_000004,20220810,2,17,3,106,번영로,0,0,70.0,...,0.0,0,부록교 시종점,33.501477,126.569223,0,봉개교 시종점,33.496863,126.581230,0
5,TEST_000005,20220803,2,1,2,103,일반국도12호선,0,0,60.0,...,0.0,0,외도1동,33.493036,126.430946,0,외도연대마을,33.492699,126.428241,0
6,TEST_000006,20220822,3,4,1,107,수영장길,0,0,60.0,...,0.0,0,제2오라교,33.498363,126.516407,0,한국병원앞,33.499973,126.516469,1
7,TEST_000007,20220809,6,5,2,107,-,0,0,80.0,...,0.0,0,해안교,33.466433,126.454583,0,해안교,33.466607,126.454022,0
8,TEST_000008,20220818,1,5,2,106,지방도1136호선,0,0,60.0,...,0.0,0,서귀포시산림조합,33.260382,126.513153,0,청룡개발,33.260513,126.514851,0
9,TEST_000009,20220828,4,3,3,107,제2거로교,0,0,70.0,...,0.0,3,제2거로교,33.508712,126.558703,0,제2거로교,33.508464,126.558231,0


In [36]:
# 주말(토, 일) - 요일컬럼으로 알수있음, 연휴(3일이상, 시작, 중간, 끝), 단독 굥휴일를 체크하는 컬럼을 만듦?
# pip install pytimeker 0.1.0

from pytimekr import pytimekr

kr = pytimekr.

In [69]:
# 1월 1일, 설날, 삼일절, 어린이날, 부처님오신날, 현충일, 광복절, 추석, 개천절, 한글날, 크리스마스
# 대체 : 설, 추석, 어린이날, 한글날, 광복절, 삼일절
int(''.join(str(kr[0]).split('-')))

20220131

In [41]:
from datetime import datetime, date
# 0이 월
today = datetime(2022, 10, 26)   
print(today.weekday())


2


In [72]:
int(train['base_date'][0])

20220623

In [70]:
da = str(train['base_date'][0])

year = int(da[: 4])
month = int(da[4 : 6])
day = int(da[6 :])

today = datetime(year, month, day)   
print(today.weekday())

3


In [90]:
# 20221022 정수형태로 리스트에 정리

df_year = 2022

chuseok = []
lunar_newyear = []
y_list = []
x_list = []

# 대체 있음
c = pytimekr.red_days(pytimekr.chuseok(year = df_year))                     # 추석
l = pytimekr.red_days(pytimekr.lunar_newyear(year = df_year))               # 설날

for idx, item in enumerate(c):
    chuseok.append(int(''.join(str(item).split('-'))))
    
for idx, item in enumerate(l):
    lunar_newyear.append(int(''.join(str(item).split('-'))))


y_list.append(int(''.join(str(pytimekr.hangul(year = df_year)).split('-'))))                      # 한글날
y_list.append(int(''.join(str(pytimekr.children(year = df_year)).split('-'))))                    # 어린이날
y_list.append(int(''.join(str(pytimekr.independence(year = df_year)).split('-'))))                # 광복절
y_list.append(int(''.join(str(pytimekr.samiljeol(year = df_year)).split('-'))))                   # 삼일절


# 대체 없음
x_list.append(int(''.join(str(pytimekr.memorial(year = df_year)).split('-'))))                    # 현충일
x_list.append(int(''.join(str(pytimekr.buddha(year = df_year)).split('-'))))                      # 석가탄신일
x_list.append(int(''.join(str(pytimekr.constitution(year = df_year)).split('-'))))                # 제헌절
x_list.append(int(''.join(str(pytimekr.christmas(year = df_year)).split('-'))))                   # 크리스마스
x_list.append(int(''.join(str(pytimekr.newyear(year = df_year)).split('-'))))                     # 1월 1일

In [91]:
holidays_dic = {
                    'chuseok' : chuseok,
                    'lunar_newyear' : lunar_newyear,
                    'y_list' : y_list,
                    'x_list' : x_list
}

In [92]:
holidays_dic

{'chuseok': [20220909, 20220910, 20220911],
 'lunar_newyear': [20220131, 20220201, 20220202],
 'y_list': [20221009, 20220505, 20220815, 20220301],
 'x_list': [20220606, 20220508, 20220717, 20221225, 20220101]}

In [ ]:
# 날짜를 뽑아서 날짜를 공휴일과 비교?
# 뽑은 공휴일을 날짜에 비교?

day_data = 20220131

for key in list(holidays_dic.keys()):
        if day_data in holidays_dic[key]:
            if key == 'x_list':
                print('대체 X 단독 공휴일')
            elif key == 'y_list':
                if 요일데이터 == 주말:
                    월요일을 대체공휴일로
                print('대체 Y 단독 공휴일')
            else:
                if 요일데이터 == holidays_dic[key][1]:
                    토요일이면 월요일을 대체공휴일로
                    월요일이면 화요일을 대체공휴일로
                
                holidays_dic[key][0]과 holidays_dic[key][2]는 연휴 시작과 끝?
            
        요일데이터
                